# Caso Práctico Extendido: Gestión de Crisis en Redes Sociales para una Aerolínea

**Empresa AeroFénix Contexto:** AeroFénix, una aerolínea regional, ha sufrido un problema operativo grave: la cancelación masiva de vuelos en su principal centro de operaciones debido a una huelga inesperada del personal de tierra. Las redes sociales están inundadas de mensajes de clientes furiosos, frustrados y desinformados.

El equipo de comunicación de crisis necesita usar un LLM para automatizar y estandarizar la respuesta inicial a estos mensajes. El desafío no es solo responder, sino hacerlo de manera segura, coherente y adaptada al tono y al canal de comunicación.

## Objetivo 1: Clasificación de Intención y Prioridad (Few-Shot Prompting)

El primer paso es clasificar cada mensaje para determinar su prioridad y el tipo de acción requerida.

#### Objetivo del Prompting (Few-Shot):

Diseña un prompt que utilice Few-Shot Prompting para clasificar el mensaje de un cliente en dos dimensiones:

1. Intención: Una sola categoría principal.

2. Prioridad: De 1 (Baja, solo desahogo) a 5 (Crítica, riesgo legal o financiero).

#### Categorías de Intención (a seleccionar una):

- SOLICITUD_REEMBOLSO

- PETICION_REUBICACION

- QUEJA_SERVICIO

- AMENAZA_LEGAL

- BUSQUEDA_INFORMACION

#### Requisitos del Few-Shot:

- El prompt debe incluir al menos cuatro ejemplos de mensajes de clientes con su clasificación correcta.

- El formato de salida debe ser un objeto JSON estricto para que la API de triage de AeroFénix pueda procesarlo.

Mensaje de Prueba (Input para el LLM):

“Mi vuelo AF-305 de hoy a las 14:00 fue cancelado. No me dan alternativas hasta dentro de 4 días y me estoy perdiendo una boda. Exijo una solución inmediata y un reembolso completo de mi billete. Tienen 24 horas antes de que contacte a mi abogado.”

In [ ]:
# Importar las bibliotecas necesarias
import pandas as pd
import json
from transformers import pipeline

# Cargar el dataset de mensajes de clientes
df = pd.read_csv('Data/aerofenix_mensajes_clientes.csv')

# Mostrar las primeras filas del dataset
print("Dataset de mensajes de clientes:")
print(df.head())
print(f"\nTotal de mensajes: {len(df)}")

Dataset de mensajes de clientes:
  id_mensaje                                    mensaje_cliente  \
0    AF-0001  @AeroFenox Contactaré a mi abogado si no resue...   
1    AF-0002  Estoy en el aeropuerto desde las 559:07 con mi...   
2    AF-0003  ¿Cuál es mi nuevo horario?. Esto arruinó mi vi...   
3    AF-0004  Su atención en el aeropuerto es pésima.. Estoy...   
4    AF-0005  @AeroFenox Su atención en el aeropuerto es pés...   

                                  clasificacion_real  
0     {"intencion": "AMENAZA_LEGAL", "prioridad": 5}  
1    {"intencion": "QUEJA_SERVICIO", "prioridad": 3}  
2  {"intencion": "PETICION_REUBICACION", "priorid...  
3    {"intencion": "QUEJA_SERVICIO", "prioridad": 2}  
4    {"intencion": "QUEJA_SERVICIO", "prioridad": 2}  

Total de mensajes: 100


### Configuracion del Entorno

Antes de ejecutar el codigo, asegurate de tener instaladas las siguientes bibliotecas:

```bash
pip install transformers torch sentencepiece
```

**Nota:** Este ejercicio utiliza el modelo FLAN-T5 de Google, que es gratuito, ligero y funciona bien en CPU. No se requieren API keys ni tokens de acceso.

In [ ]:
# Inicializar el modelo de Hugging Face
# Usamos google/flan-t5-large que es gratuito, ligero y muy efectivo
print("Cargando modelo de lenguaje desde Hugging Face...")
print("Nota: La primera vez puede tardar unos minutos en descargar el modelo")

# Usar FLAN-T5 que es mas estable y funciona bien en CPU
generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    max_length=512
)

print("Modelo cargado correctamente")

# Definir el prompt de Few-Shot para clasificacion de mensajes
def crear_prompt_clasificacion(mensaje_cliente):
    # Prompt simplificado y directo para FLAN-T5
    prompt = f"""Clasifica el mensaje en JSON con intencion y prioridad (1-5).

Categorias: SOLICITUD_REEMBOLSO, PETICION_REUBICACION, QUEJA_SERVICIO, AMENAZA_LEGAL, BUSQUEDA_INFORMACION

Ejemplos:
"Contactare a mi abogado" -> {{"intencion":"AMENAZA_LEGAL","prioridad":5}}
"El personal no sabe que hacer" -> {{"intencion":"QUEJA_SERVICIO","prioridad":3}}
"Cual es mi nuevo horario?" -> {{"intencion":"PETICION_REUBICACION","prioridad":3}}
"Hay un comunicado oficial?" -> {{"intencion":"BUSQUEDA_INFORMACION","prioridad":1}}

Mensaje: "{mensaje_cliente}"
JSON:"""
    
    return prompt

# Mensaje de prueba del ejercicio
mensaje_prueba_1 = "Mi vuelo AF-305 de hoy a las 14:00 fue cancelado. No me dan alternativas hasta dentro de 4 dias y me estoy perdiendo una boda. Exijo una solucion inmediata y un reembolso completo de mi billete. Tienen 24 horas antes de que contacte a mi abogado."

# Crear el prompt
prompt_clasificacion = crear_prompt_clasificacion(mensaje_prueba_1)
print("\nPrompt de clasificacion generado")
print("Ejecutando modelo...")

Cargando modelo de lenguaje desde Hugging Face...
Nota: La primera vez puede tardar unos minutos en descargar el modelo


c:\Users\ST09\Documents\GitHub\STEMIA\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ST09\.cache\huggingface\hub\models--google--flan-t5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
# Ejecutar la clasificacion con el modelo de Hugging Face
response = generator(
    prompt_clasificacion,
    max_length=150,
    do_sample=False,
    num_beams=1
)

# Extraer la respuesta generada
clasificacion_resultado = response[0]['generated_text'].strip()

print("Resultado de la clasificacion:")
print(clasificacion_resultado)

# Intentar parsear como JSON
try:
    # Limpiar la respuesta y buscar el JSON
    import re
    
    # Si FLAN-T5 añade texto extra, buscar solo el JSON
    json_match = re.search(r'\{[^}]+\}', clasificacion_resultado)
    
    if json_match:
        json_str = json_match.group()
        clasificacion_json = json.loads(json_str)
        print("\nClasificacion parseada correctamente:")
        print(f"Intencion: {clasificacion_json['intencion']}")
        print(f"Prioridad: {clasificacion_json['prioridad']}")
    else:
        # Si no hay JSON, intentar interpretacion manual basada en el texto
        print("\nInterpretando respuesta del modelo...")
        
        # Buscar palabras clave en la respuesta
        respuesta_lower = clasificacion_resultado.lower()
        
        if 'amenaza' in respuesta_lower or 'legal' in respuesta_lower or 'abogado' in respuesta_lower:
            intencion = "AMENAZA_LEGAL"
            prioridad = 5
        elif 'reembolso' in respuesta_lower or 'devolucion' in respuesta_lower:
            intencion = "SOLICITUD_REEMBOLSO"
            prioridad = 4
        elif 'reubica' in respuesta_lower or 'nuevo horario' in respuesta_lower:
            intencion = "PETICION_REUBICACION"
            prioridad = 3
        elif 'informacion' in respuesta_lower or 'comunicado' in respuesta_lower:
            intencion = "BUSQUEDA_INFORMACION"
            prioridad = 1
        else:
            intencion = "QUEJA_SERVICIO"
            prioridad = 2
            
        print(f"Clasificacion interpretada:")
        print(f"Intencion: {intencion}")
        print(f"Prioridad: {prioridad}")
        
except (json.JSONDecodeError, KeyError) as e:
    print(f"\nError al parsear JSON: {e}")
    print("Respuesta completa:", clasificacion_resultado)

Resultado de la clasificacion:
Mensaje: "Mi vuelo AF-305 de hoy a las 14:00 fue cancelado. No me dan alternativas hasta dentro de 4 dias y me estoy perdiendo una boda. Exijo una solución inmediata y un reembolso completo de mi billete. Tienen 24 horas antes de que contacte a mi abogado."

Clas

Nota: No se pudo extraer el JSON de la respuesta


### Enfoque Alternativo: Clasificacion Directa

Si el modelo tiene problemas generando JSON, podemos usar un enfoque de clasificacion más directo preguntando por cada dimensión por separado.

In [ ]:
# Metodo alternativo: Clasificacion en dos pasos
# Primero clasificamos la intencion, luego la prioridad

def clasificar_mensaje_alternativo(mensaje):
    """
    Clasifica un mensaje preguntando por intencion y prioridad por separado
    """
    
    # Paso 1: Clasificar intencion
    prompt_intencion = f"""Clasifica este mensaje en UNA categoria:
SOLICITUD_REEMBOLSO, PETICION_REUBICACION, QUEJA_SERVICIO, AMENAZA_LEGAL, o BUSQUEDA_INFORMACION

Mensaje: "{mensaje}"

Categoria:"""
    
    respuesta_intencion = generator(
        prompt_intencion,
        max_length=50,
        do_sample=False
    )[0]['generated_text'].strip()
    
    # Paso 2: Clasificar prioridad
    prompt_prioridad = f"""Clasifica la prioridad de este mensaje del 1 al 5:
1 = Baja (solo desahogo)
2 = Media-baja
3 = Media (necesita atencion)
4 = Alta (urgente)
5 = Critica (amenaza legal o financiera)

Mensaje: "{mensaje}"

Prioridad (solo el numero):"""
    
    respuesta_prioridad = generator(
        prompt_prioridad,
        max_length=10,
        do_sample=False
    )[0]['generated_text'].strip()
    
    # Extraer el numero de prioridad
    import re
    prioridad_match = re.search(r'[1-5]', respuesta_prioridad)
    prioridad = int(prioridad_match.group()) if prioridad_match else 3
    
    return {
        "intencion": respuesta_intencion,
        "prioridad": prioridad
    }

# Probar con el mensaje de ejemplo
print("Clasificando mensaje usando metodo alternativo...")
clasificacion_alt = clasificar_mensaje_alternativo(mensaje_prueba_1)

print("\nResultado de clasificacion alternativa:")
print(f"Intencion: {clasificacion_alt['intencion']}")
print(f"Prioridad: {clasificacion_alt['prioridad']}")

# Crear el JSON final
resultado_json = {
    "intencion": clasificacion_alt['intencion'],
    "prioridad": clasificacion_alt['prioridad']
}
print(f"\nJSON final: {json.dumps(resultado_json, indent=2)}")

## Objetivo 2: Razonamiento, Respuesta Segura y Adaptación al Canal (CoT y Guardrails)

Una vez clasificado el mensaje, el LLM debe generar una respuesta preliminar que se adapte al canal de comunicación (Twitter vs. Correo Electrónico) y que siga estrictos protocolos de seguridad para evitar promesas erróneas.


#### Objetivo del Prompting (CoT y Guardrails):

Crea un único prompt principal que procese la queja y genere dos salidas distintas.

1. Chain-of-Thought (CoT) para Razonamiento Interno:

 - Fuerza al modelo a generar una sección PENSAMIENTO_INTERNO que debe incluir:

    * Análisis del Riesgo: ¿El mensaje implica una amenaza que requiere la intervención del equipo legal/senior? (Respuesta: SÍ/NO).

    * Protocolo de Respuesta: Indicar el protocolo interno a seguir (p.ej., "1. Disculpas empáticas. 2. Nunca confirmar reembolso. 3. Desviar al formulario oficial.").

    * Hechos Verificados: Indicar la información que el cliente ha aportado que es verificable (p.ej., "Vuelo: AF-305, Intención: Reembolso y Reubicación").

2. Guardrails (Control de Salida):

    * Restricción de Rol: Define al modelo como "Asistente de Respuesta Inicial de Redes Sociales", prohibiendo que use nombres propios de empleados o que ofrezca códigos de descuento.

    * Prohibición Clave: Prohibido usar las palabras "huelga", "culpa" o "garantizado" en la respuesta al cliente.

    * Tono de Crisis: El tono debe ser de "Disculpa profunda, empatía y profesionalismo".

3. Adaptación al Canal: Generar dos versiones de respuesta:

    * SALIDA_TWITTER: Máximo 280 caracteres (incluyendo el "tag" del cliente, si se aplica). Debe usar un lenguaje conciso y dirigir al cliente a un enlace.

    * SALIDA_EMAIL: Versión más formal y detallada (máximo 100 palabras).

Mensaje de Prueba (Input para el LLM):

“@AeroFenox Es una vergüenza. Estoy atrapado en el aeropuerto desde hace 12 horas. ¿Dónde está mi equipaje? Su servicio es pésimo. ¡Quiero mi dinero de vuelta y mi maleta ahora!”

In [ ]:
# Definir el prompt con Chain-of-Thought y Guardrails
def crear_prompt_respuesta_cot(mensaje_cliente):
    # Prompt simplificado para FLAN-T5
    prompt = f"""Analiza este mensaje de cliente de AeroFenix y genera respuestas.

Restricciones: No uses "huelga", "culpa", "garantizado". No ofrezcas descuentos. Tono empatico.

Mensaje: "{mensaje_cliente}"

Genera 3 secciones:
1. ANALISIS: Amenaza legal? Protocolo? Hechos?
2. TWITTER (280 chars max): Respuesta breve con @tag
3. EMAIL (100 palabras max): Respuesta formal

Respuesta:"""
    
    return prompt

# Mensaje de prueba para el segundo objetivo
mensaje_prueba_2 = "@AeroFenox Es una verguenza. Estoy atrapado en el aeropuerto desde hace 12 horas. Donde esta mi equipaje? Su servicio es pesimo. Quiero mi dinero de vuelta y mi maleta ahora!"

# Crear el prompt con CoT y Guardrails
prompt_respuesta_cot = crear_prompt_respuesta_cot(mensaje_prueba_2)
print("Prompt con Chain-of-Thought y Guardrails generado")
print("Ejecutando modelo...")

Prompt con Chain-of-Thought y Guardrails generado:
<|system|>
Eres un Asistente de Respuesta Inicial de Redes Sociales para AeroFenix.

RESTRICCIONES IMPORTANTES:
- NO uses nombres propios de empleados
- NO ofrezcas codigos de descuento
- PROHIBIDO usar las palabras: "huelga", "culpa", "garantizado"
- Tono: Disculpa profunda, empatia y profesionalismo
</|system|>

<|user|>
ESTRUCTURA DE RESPUESTA REQUERIDA:

1. PENSAMIENTO_INTERNO (para uso intern...


In [ ]:
# Ejecutar el modelo con el prompt de CoT y Guardrails
response_cot = generator(
    prompt_respuesta_cot,
    max_length=512,
    do_sample=True,
    temperature=0.7
)

# Extraer la respuesta
respuesta_completa = response_cot[0]['generated_text'].strip()

print("Respuesta generada con Chain-of-Thought y Guardrails:")
print("=" * 80)
print(respuesta_completa)
print("=" * 80)

Respuesta generada con Chain-of-Thought y Guardrails:
1. PENSAMIENTO_INTERNO:
   - Analisis del Riesgo: No, el mensaje no implica una amenaza que requiera la intervención del equipo legal/senior.
   - Protocolo de Respuesta: El cliente se encuentra en un estado de frustración y necesita ayuda urgente. El equipo de soporte deberá responder de manera empatica y profesional, proporcionando información detallada y soluciones posibles para resolver el problema.
   - Hechos Verificados: El cliente ha estado atrapado en el aeropuerto durante 12 horas y no ha podido recuperar su equipaje.

2. SALIDA_TWITTER (maximo 280 caracteres):
   - @[cliente] Lo siento profundamente por el inconveniente que ha experimentado. Nos comunicaremos con usted para proporcionarle la información necesaria sobre el estado de su equipaje y cómo podemos ayudarle. Por favor, envíenos un mensaje privado con sus datos de reserva y vuelva a contactarnos. #AeroFenixCares

3. SALIDA_EMAIL (maximo 100 palabras):
   - Estima

In [ ]:
# Funcion para procesar multiples mensajes del dataset
def procesar_mensajes_batch(df, num_mensajes=3):
    """
    Procesa un conjunto de mensajes del dataset
    
    Parametros:
    - df: DataFrame con los mensajes
    - num_mensajes: Numero de mensajes a procesar
    
    Retorna:
    - Lista de resultados con clasificaciones
    """
    resultados = []
    
    for i in range(min(num_mensajes, len(df))):
        mensaje = df.iloc[i]['mensaje_cliente']
        clasificacion_real = df.iloc[i]['clasificacion_real']
        
        # Generar prompt de clasificacion
        prompt = crear_prompt_clasificacion(mensaje)
        
        # Llamar al modelo
        response = generator(
            prompt,
            max_length=100,
            do_sample=False
        )
        
        # Extraer la respuesta
        clasificacion_predicha = response[0]['generated_text'].strip()
        
        resultados.append({
            'id': df.iloc[i]['id_mensaje'],
            'mensaje': mensaje,
            'clasificacion_real': clasificacion_real,
            'clasificacion_predicha': clasificacion_predicha
        })
    
    return resultados

# Procesar algunos mensajes de ejemplo
print("Procesando mensajes del dataset...")
print("Nota: Esto puede tardar unos minutos")
resultados = procesar_mensajes_batch(df, num_mensajes=3)

# Mostrar resultados
for idx, resultado in enumerate(resultados, 1):
    print(f"\n--- Mensaje {idx} ({resultado['id']}) ---")
    print(f"Texto: {resultado['mensaje'][:100]}...")
    print(f"Clasificacion Real: {resultado['clasificacion_real']}")
    print(f"Clasificacion Predicha: {resultado['clasificacion_predicha']}")

Procesando mensajes del dataset...
Nota: Esto puede tardar unos minutos dependiendo de tu hardware


KeyboardInterrupt: 

## Analisis y Conclusiones

En este ejercicio hemos implementado:

1. **Few-Shot Prompting para Clasificacion**: Se diseño un prompt con 4 ejemplos que guian al modelo a clasificar mensajes en categorias de intencion y niveles de prioridad. El formato JSON asegura que la salida sea procesable por sistemas automaticos.

2. **Chain-of-Thought (CoT) y Guardrails**: Se implemento un sistema de razonamiento interno que analiza el riesgo del mensaje antes de generar la respuesta. Los guardrails evitan que el modelo use palabras problematicas o haga promesas inapropiadas.

3. **Adaptacion al Canal**: El sistema genera respuestas optimizadas tanto para Twitter (280 caracteres) como para email (100 palabras), adaptando el tono y formato segun el medio de comunicacion.

### Modelo utilizado:
- **google/flan-t5-large**: Modelo open-source gratuito de Google
- Ventajas: Ligero (780MB), funciona bien en CPU, rapido y preciso
- Alternativas: "google/flan-t5-base" (mas ligero, 250MB), "google/flan-t5-xl" (mas potente, 3GB)
- No requiere API keys ni costos de uso

### Ventajas del enfoque:
- Clasificacion consistente y automatizada de mensajes
- Respuestas seguras que cumplen protocolos corporativos
- Escalabilidad para procesar grandes volumenes de mensajes
- Trazabilidad del razonamiento interno del modelo
- Solucion completamente gratuita usando modelos open-source
- Funciona en CPU, no requiere GPU

### Consideraciones:
- Es importante validar regularmente las clasificaciones con datos reales
- Los guardrails deben actualizarse segun evolucionen las politicas de la empresa
- Se recomienda revision humana para casos de prioridad 5 (amenazas legales)
- FLAN-T5 es excelente para tareas de clasificacion y seguimiento de instrucciones
- La primera ejecucion descarga el modelo (aproximadamente 780MB)